In [17]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
tfe.enable_eager_execution()

In [13]:
embed_size = 10
window_size = 3
dict_size = 5
filter_size = window_size*embed_size

def input_fn():
    ids = tf.constant([[5, 6, 10, 9, 2], [5, 6, 10, 60, 2]])
    label_ids = tf.constant([[4, 22, 163, 649, 2], [4, 22, 163, 649, 2]])
    return {'ids': ids,
            'init_label': label_ids[:, :window_size],
            'label_length': int(label_ids.shape[1])-window_size
            }, label_ids[:, window_size:]

In [31]:
ids = tf.constant([[5, 6, 10, 9, 2], [5, 6, 10, 60, 2]])
label_ids = tf.constant([[4, 22, 163, 649, 2], [4, 22, 163, 649, 2]])
init_label = label_ids[:, :window_size]
label_legnth = int(label_ids.shape[1]) - window_size
label_ids[: window_size:]

<tf.Tensor: id=245, shape=(2, 5), dtype=int32, numpy=
array([[  4,  22, 163, 649,   2],
       [  4,  22, 163, 649,   2]], dtype=int32)>

In [41]:
tf.nn.embedding_lookup(embeddings, ids)

InvalidArgumentError: indices[0,0] = 5 is not in [0, 5) [Op:ResourceGather] name: embedding_lookup/embedding_lookup//

In [35]:
embeddings = tfe.Variable(tf.truncated_normal([dict_size, embed_size]), trainable=False)
#print(embeddings.numpy())

# embedding
input_embeds = tf.nn.embedding_lookup(embeddings, ids)

# input_flat = tf.layers.flatten(input_embeds)
# input_flat = tf.expand_dims(input_flat, -1)

# init_label = tf.nn.embedding_lookup(embeddings, features['init_label'])
# init_label = tf.layers.flatten(init_label)
# init_label = tf.expand_dims(init_label, -1)

InvalidArgumentError: indices[0,0] = 5 is not in [0, 5) [Op:ResourceGather] name: embedding_lookup/embedding_lookup//

In [36]:
def model_fn(mode, features, labels):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT

    embeddings = tf.Variable(tf.truncated_normal([dict_size, embed_size]), trainable=False)

    # embedding
    input_embeds = tf.nn.embedding_lookup(embeddings, features['ids'])

    input_flat = tf.layers.flatten(input_embeds)
    input_flat = tf.expand_dims(input_flat, -1)

    init_label = tf.nn.embedding_lookup(embeddings, features['init_label'])
    init_label = tf.layers.flatten(init_label)
    init_label = tf.expand_dims(init_label, -1)

    if not PREDICT:
        label_onehot = tf.one_hot(labels, depth=dict_size, dtype=tf.float32)

    # encoder
    encoder_conv = tf.layers.conv1d(
            inputs=input_flat,
            filters=2*embed_size,
            kernel_size=filter_size,
            strides=embed_size,
            padding='same')

    encoder_glu = encoder_conv[:, :, embed_size:]*tf.nn.sigmoid(encoder_conv[:, :, :embed_size])

    #decoder
    next_ids = []
    outs = []
    for l in range(features['label_length']):
        if l == 0: decoder_input = init_label

        decoder_conv = tf.layers.conv1d(
                inputs=init_label,
                filters=2*embed_size,
                kernel_size=filter_size,
                strides=embed_size)


        decoder_glu = decoder_conv[:, :, embed_size:]*tf.nn.sigmoid(decoder_conv[:, :, :embed_size])

        tiled_decoder_glu = tf.tile(decoder_glu, [1, int(encoder_glu.shape[1]), 1])


        dot_prod = tf.matmul(encoder_glu, decoder_glu, transpose_b=True)

        attention = tf.nn.softmax(dot_prod)

        z_plus_e = encoder_glu + input_embeds

        tiled_attention = tf.tile(attention, [1, 1, embed_size])

        c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
        decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])

        logits = tf.layers.dense(c+decoder_glu, dict_size)

        out = tf.nn.softmax(logits)

        next_id = tf.argmax(out, axis=1)
        next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
        next_embeds = tf.expand_dims(next_embeds, -1)
        decoder_input = tf.concat([decoder_input[:, embed_size:], next_embeds], axis=1)

        next_ids.append(next_id)
        outs.append(logits)

    outs = tf.stack(outs, axis=1)
    next_ids = tf.stack(next_ids, axis=1)

    if TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.softmax_cross_entropy(label_onehot, outs)
        train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss, global_step)
        estimator_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                train_op=train_op,
                loss=loss)

    elif EVAL:
        loss = tf.losses.softmax_cross_entropy(label_onehot, outs)
        eval_metric_ops = {'acc': tf.metrics.accuracy(labels, next_ids)}
        estimator_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=loss,
                eval_metric_ops=eval_metric_ops)

    elif PREDICT:
        estimator_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={'prediction': next_ids})

    else:
        raise Exception('estiamtor spec is invalid')

    return estimator_spec


def main(_):
    fairseq = tf.estimator.Estimator(model_fn)
    fairseq.train(input_fn, steps=10000)
    fairseq.evaluate(input_fn, steps=10)

    pred = fairseq.predict(input_fn)
    for p in pred:
        print('prediction: {}'.format(p['prediction']))

if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.app.run()

FATAL Flags parsing error: Unknown command line flag 'f'
Pass --helpshort or --helpfull to see help on flags.


SystemExit: 1

/Users/naver/tf_nightly/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
